# **TFRecord and `tf.Example`**

**Learning objectives**

1. Understand the TFRecord format for storing data
2. Understand the `tf.Example` message type
3. Read and write a `TFRecord` file

## **Introduction**

In this notebook, you create, parse and use the `tf.Example` message, and then serialise, write, and read `tf.Example` messages to and from `.tfrecord` files. To read data efficiently it can be helpful to serialise your data and store it in a set of files (100-200MB each) that can each be read linearly. This is especially true if the data is being streamed over a network. This can also be useful for caching any data-preprocessing.